# Document Classification - Basic Steps

The goal of this tutorial is to perform text classification "from scratch", i.e., without using an implementation of a classifier provided by existing packages. This gives a better intuition how classifiers work.

To this end, the tutorial motivates and implements one of the simplest classifiers, the **Naive Bayes Classifier**. The basic idea of this classifier is to answer the questions: Given a document consisting of a set of words, what is the probability it belongs to a certain class. 

## Import all important packages

In [1]:
import numpy as np
import pandas as pd
import random

from nltk import bigrams
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from utils.nlputil import preprocess_text

## Load dataset

### Read data files

Use Python package `pandas` to read files. This dataset consists of 2 text files, one containing 5,331 positive sentences and the other 5,331 negative sentiences.

In [2]:
df_sent_pos = pd.read_csv('data/sentence-polarity-dataset/sentence-polarity.pos', sep='\t', header=None)
df_sent_neg = pd.read_csv('data/sentence-polarity-dataset/sentence-polarity.neg', sep='\t', header=None)

### Create internal representation of dataset

For the training and testing, we want two lists, one containing all sentences and another containing the respective labels (here `0` representing negative ans `1` representing positive sentences). Note that there is nothing special about labelling the classes. We could equally use the strings `"negative"` and `"positive"`. Some additional explanations:

- The list method `A.extend(B)` attaches list `B` to list `A`

- `[0]*len(df_sent_neg)` creates a a list `[0, 0, 0, 0, 0, ...]` of length $N$ with $N$ being the number of, here, negative sentences

- `np.array(A)` converts a normal n-dimensional Python list into an n-dimensional numpy array (see `import numpy as np` above). It is not crucial since methods for training and test take both standard lists and numpy arrays as input, but numpy arrays come with a long list of useful functions and features.

In [3]:
# Create a list for all sentences and ad the sentences from both read files
sentences_raw = []
sentences_raw.extend(df_sent_neg[0].tolist())
sentences_raw.extend(df_sent_pos[0].tolist())

sentences = [''] * len(sentences_raw)
for idx, doc in enumerate(sentences_raw):
    #sentences[idx] = preprocess_text(doc)
    sentences[idx] = doc
  
# Create a list for all lables
polarities = []
polarities.extend([0]*len(df_sent_neg))
polarities.extend([1]*len(df_sent_pos))

# Convert from lists to numpy arrays
sentences = np.array(sentences)
polarities = np.array(polarities)

Right now, the dataset contains 5,331 positive sentences followed by 5,331 negative sentence. Before we can split the dataset into training and test data, we first have to shuffle the order to ensure a balanced dataset to in turn ensure a balanced training and test data size. Some additional explanations:

- `combined = list(zip(sentences, polarities))`: We have to lists containing the sentences and the labels. Of course, we have to ensure the both list are shuffled the same way so that each label keeps associated with the same sentence. The `zip()` method accomplishes this, both zipping and unzipping.

- `random.seed(int)` (optional): the `shuffle()` method does not truly randomize the order of the elements of a list, but generates a "pseudo-randomized" order. This in turn allows that, by providing a fixed $seed$, we can ensure that shuffling always returns the same "random" order. This makes the whole process deterministic and can be useful find problems.

In [4]:
combined = list(zip(sentences, polarities))

#random.seed(1) (optional)
random.shuffle(combined)

# split the "zipped" list into the two lists of sentences and labels/polarities
sentences[:], polarities[:] = zip(*combined)

### Generate training and test data

Given 100% of the data, a common way is to split it into 80% (90%) of training data and 20% (10%) of test data. The training is only done using the training and the test only using the testing data, respectively. To make meaningful statements about the effectiveness of the classifiers requires (at least) the the testing is done using data the classifiers has never seen before. Some additional explanations:

- `A[:n]` returns the first $n$ elements of list A

- `A[n:]` returns all the elements after the $n$-th elemnts of list A

In [5]:
# Let's go for a 80%/20% split -- you can change the value anf see its effects
train_test_ratio = 0.8

# Calculate the size of the training data (the size of the dest data is also implicitly given)
train_set_size = int(train_test_ratio * len(sentences))

# Split data and labels into training and test data with respect to the size of the test data
X_train, X_test = sentences[:train_set_size], sentences[train_set_size:]
y_train, y_test = polarities[:train_set_size], polarities[train_set_size:]

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))

Size of training set: 8529
Size of test: 2133


## Implementing a Naive Bayes Classifier

The Naive Bayes Classifier classifies documents - given by a set of words $\{w_1, w_2, ..., w_n\}$ - by caluclating the conditional probablities 

$$P(c_i\ |\  w_1,w_2,...,w_n)$$ 

for all classes $c_i$, and picking the class with the highest conditional probability. For example, one would assume that $P(c_{pos}\ |\  happy,luck,...,vacation)$ has a higher value that $P(c_{pos}\ |\  accident,bad,...,traffic)$.

Using Bayes' Therorm, we can write:

$$P(c_i\ |\  w_1,w_2,...,w_n) = \frac{P(w_1,w_2,...,w_n \ |\ c_i)  \cdot P(c_i)}{P(w_1,w_2,...,w_n)}$$

$P(c_i)$ is called the prior probability of class $c_i$ and simply reflects the distribution of the different classes in the set of documents. For example, if our dataset of positive and negative documents (i.e., 2 classes) contains 55% positive sentences, $P(c_{pos})=0.55$ and $P(c_{neg})=0.45$.

We can further simply this calculation. In the end, we are only interested which has the higher probability, $P(c_{pos}\ |\  w_1,w_2,...,w_n)$ or $P(c_{neg}\ |\  w_1,w_2,...,w_n)$. The absolute values are not important. As such, we can ignore the denominator $P(w_1,w_2,...,w_n)$ since it does not depend on the class $c_i$. We therefore can write:

$$P(c_i\ |\  w_1,w_2,...,w_n) \propto P(w_1,w_2,...,w_n \ |\ c_i)  \cdot P(c_i)$$

Note that we no longer can user "$=$", since $P(c_i\ |\  w_1,w_2,...,w_n)$ is only proptional to the product on the right-hand side.

In general, $P(w_1,w_2,...,w_n \ |\ c_i)$ is difficult to calculate. This is where the "Naive Bayes" assumption comes in - that is, we assume that the all words $w_1,w_2,...,w_n$ are independet from each other. In general, this assumption does not hold. For example, documents containing "birthday" often also contain "happy". But it turns out that in practice this assumption hardly affects the results. We can now write:

$$P(c_i\ |\  w_1,w_2,...,w_n) \propto P(w_1\ |\ c_i)  \cdot P(w_2\ |\ c_i)\cdot ...  \cdot P(w_n\ |\ c_i)  \cdot P(c_i) =  P(c_i)\cdot \prod P(w_i\ |\ c_i)$$

with $P(w_i\ |\ c_i)$ being the probablity that of finding word $w_i$ in a document of class $c_i$. In other words, we can say:

$$P(w_i\ |\ c_i) = \frac{\#occurrences\ of\ w_i\ in\ c_i}{\#words\ in\ c_i}$$

These values are easy to calculate for a given set of documents. It's all about counting words, that's it.

While this works fine in theory, in practice, two concerns need to be addressed. Firstly, most $P(w_i\ |\ c_i)$ are very small probabilities. Thus if a document contains hundreds or even thousands of words, hundreds or thousands of small numbers need to be multiplied. The result is then to small to be represented in a computer and rounded to 0. To avoid this, we calculate the **log probability**. Since the logarithm is a monotonic function, it won't affect the final decision for the classification. Using the rules of logarithm, we can write:

$$\log{P(c_i\ |\  w_1,w_2,...,w_n)} \propto \log{P(c_i)}\cdot \log{\sum P(w_i\ |\ c_i)}$$

Another problem is if $P(w_i\ |\ c_i) = 0$, i.e., word $w_i$ never appeared in class $c_i$. This can easily happen if the classifier gets a document with words it has never seen before. From a mathematical perspective this is problem since $\log{0}$ is undefined. And even if without considering the logarithm, $P(w_i\ |\ c_i) = 0$ would dominate the forumala and would make $P(c_i\ |\  w_1,w_2,...,w_n) = 0$ no matter how common all other words are. The solution is to assign even unknown words with very small probability greater than 0. Without going into details, a common approach is *Laplace Smoothing*, which results in:

$$P(w_i\ |\ c_i) = \frac{(\#occurrences\ of\ w_i\ in\ c_i)\ + 1}{(\#words\ in\ c_i) + (\#words\ in\ vocabulary)}$$

Summing up, we need to calculate 3 types of information:
- size of the vocabulary
- the log probabilties $\log{P(c_i)}$
- the number of occurces of all words $w_i$ in the different classes $c_i$

The following three variable will store these information:

In [6]:
vocabulary = set()
log_class_priors = {}
token_counts = { 'pos': {}, 'neg': {} }

The following auxiliary method `get_token_counts()` takes a list of token as input and returns the number of occurences of each word/token in the token list.

In [7]:
def get_token_counts(token_list):
    token_counts = {}
    for token in token_list:
        token_counts[token] = token_counts.get(token, 0.0) + 1.0
    return token_counts

In [8]:
token_list = X_train[-1].split() # Data is preprocessed; split() is good enough

print(get_token_counts(token_list))

{'the': 1.0, 'as': 2.0, 'raised': 1.0, 'oprah': 1.0, 'chatter': 1.0, '.': 1.0, 'supposed': 1.0, 'ends': 1.0, 'be': 1.0, 'tedious': 1.0, 'to': 1.0, 'on': 1.0, 'post-feminist': 1.0, "it's": 1.0, 'of': 1.0, 'parrots': 1.0, 'up': 1.0, 'breezy': 1.0, 'but': 1.0}


The `fit()` method does the actual caluclation of the 3 types of required information.

In [9]:
def fit(X, y):
    num_data_items = len(X)
    
    # Calculate the prior log probabilites, i.e, the ration of positive and negative documents
    log_class_priors['pos'] = np.log(sum(1 for label in y if label == 1) / num_data_items)
    log_class_priors['neg'] = np.log(sum(1 for label in y if label == 0) / num_data_items)
    
    # The whole loop essentially just counts the words for each class
    for doc, label in zip(X, y):
        polarity_class = 'pos' if label == 1 else 'neg'
        # Get token token counts for the current document
        counts = get_token_counts(doc.split())
        for token, count in counts.items():
            # Remember vocabulary so we can handle unknown tokens later
            # It's a set, so no harm to add multiple times
            vocabulary.add(token)
            # If the token is not yet in the dictionary, initialize count with 0
            if token not in token_counts[polarity_class]:
                token_counts[polarity_class][token] = 0.0
            # Update token count
            token_counts[polarity_class][token] += count   

Let's train the classifier.

In [10]:
fit(X_train, y_train)

The following illustrate the result of the calculation.

In [11]:
print(log_class_priors)
print()

print('Number of tokens in POSITIVE: {}'.format(sum(token_counts['pos'].values())))
print('Number of tokens in NEGATIVE: {}'.format(sum(token_counts['neg'].values())))
print()

token_1 = "good"
token_2 = "bad"

print('Number of occurrences of "{}" in class POSITIVE: {}'.format(token_1, token_counts['pos'].get(token_1, 0.0)))
print('Number of occurrences of "{}" in class NEGATIVE: {}'.format(token_1, token_counts['neg'].get(token_1, 0.0)))
print('Number of occurrences of "{}" in class POSITIVE: {}'.format(token_2, token_counts['pos'].get(token_2, 0.0)))
print('Number of occurrences of "{}" in class NEGATIVE: {}'.format(token_2, token_counts['neg'].get(token_2, 0.0)))

{'pos': -0.69490743449095715, 'neg': -0.69139001967907876}

Number of tokens in POSITIVE: 89664.0
Number of tokens in NEGATIVE: 88878.0

Number of occurrences of "good" in class POSITIVE: 158.0
Number of occurrences of "good" in class NEGATIVE: 137.0
Number of occurrences of "bad" in class POSITIVE: 20.0
Number of occurrences of "bad" in class NEGATIVE: 162.0


Not surprisingly, the word "good" is more likely to occur in a positive document. That "good" is still common in negative documents can be caused by documents containg phrases like "not good" or "not so good". If we consider only single words, information about negation is lost.

Nest, the method `predict{}` actually calculates $\log{P(c_i)}\cdot \log{\sum P(w_i\ |\ c_i)}$ as define about. If you look closely at individual lines, you can easily identify each parts of the calculation.

In [12]:
def predict(X, verbose=False):
    result = []
    for doc in X:
        counts = get_token_counts(doc.split())
        pos_score = 0
        neg_score = 0
        for token, _ in counts.items():
            # Ignore unknown tokens
            if token not in vocabulary: 
                continue
            # add Laplace smoothing
            log_w_given_pos = np.log( (token_counts['pos'].get(token, 0.0) + 1) / (sum(token_counts['pos'].values()) + len(vocabulary)) )
            log_w_given_neg = np.log( (token_counts['neg'].get(token, 0.0) + 1) / (sum(token_counts['neg'].values()) + len(vocabulary)) )
 
            if verbose == True:
                print("Token: {} -- positive score: {} -- negative score: {}".format(token, log_w_given_pos, log_w_given_neg))

            pos_score += log_w_given_pos
            neg_score += log_w_given_neg
 
        pos_score += log_class_priors['pos']
        neg_score += log_class_priors['neg']
 
        if verbose == True:
            print("Total positive score: {}".format(pos_score))
            print("Total negative score: {}".format(neg_score))

        if pos_score > neg_score:
            result.append(1)
        else:
            result.append(0)
    return result

In [13]:
#sample = ['good bad']
#sample = ['nice movie happy end']
#sample = ['boring flick']
sample = ['sdcsdcsdcsdc']

print("Final prediction: {}".format(predict(sample, verbose=True)))

Total positive score: -0.6949074344909572
Total negative score: -0.6913900196790788
Final prediction: [0]


At last, we can apply `predict()` over our test data.

In [14]:
y_pred = predict(X_test)

Let's see how good our classifier is:

In [15]:
print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

             precision    recall  f1-score   support

          0       0.76      0.78      0.77      1059
          1       0.78      0.76      0.77      1074

avg / total       0.77      0.77      0.77      2133

Accuracy: 0.770
